In [ ]:
import numpy as np
import networkx as nx
import networkx.algorithms.community as nx_comm
import pandas as pd
from fa2 import ForceAtlas2

In [ ]:
df_edges = pd.read_csv("data/graph.csv", sep=';')
display(df_edges)


In [ ]:
G = nx.from_pandas_edgelist(df_edges, edge_attr=True)
display(G)

In [ ]:
nx.draw_networkx(G)

In [ ]:
louvain_partitions = nx_comm.louvain_communities(G, resolution=1e-1, threshold=1e-100,seed=1)


In [ ]:
for idx in range(len(louvain_partitions)):
    print("Community number {i} has {number} members".format(i=idx,number=len(louvain_partitions[idx])))

In [ ]:
display(df_edges)

In [ ]:
print(louvain_partitions[2])

filtered_channels = pd.DataFrame(louvain_partitions[2])

In [ ]:
display(filtered_channels)

In [ ]:
filtered_channels = filtered_channels[0].sort_values(ascending=True)


In [ ]:
display(filtered_channels)

In [ ]:
filtered_channels.to_csv("data/louvain_filtered_channels.csv", sep=';', index=False)

In [ ]:
def filter_function(n):
    return n in louvain_partitions[2]

In [ ]:
sub_G = nx.subgraph_view(G, filter_node=filter_function)

In [ ]:
louvain_communities = nx_comm.louvain_communities(sub_G, resolution=0.9,threshold=1e-7, seed=1)
display(len(louvain_communities))

In [ ]:
for idx in range(len(louvain_communities)):
    print("Community number {i} has {number} members".format(i=idx,number=len(louvain_communities[idx])))

In [ ]:
display(louvain_communities[0])
# Philip de Franco, Drama Alert, IntMensOrg -->Pas très sérieux Plutôt de droite mais doit y avoir des erreurs

In [ ]:
display(louvain_communities[1])
# CNN, Vox, MSNBC, The Young Turks --> Sérieux Très à gauche

In [ ]:
display(louvain_communities[2])
# Inside Edition, Fox News, Daily Wire, Rebel News(Très à droite mais canadien) --> Très à droite 

In [ ]:
display(louvain_communities[3])
# Pat Condell (Conspiration ?), Sky News Australia, National Post  --> Canadien, Australien, British, Conspirationist

In [ ]:
display(louvain_communities[4])
# Today, China uncensored, Fox Business, The dedicated Citizen --> compliqué à classer (business et Chine ?)

In [ ]:
display(louvain_communities[5])
# Truly, ABC News, BBC News, True Crime Daily, Business Insider, NYT --> Sérieux Gauche / Leaning left / centre


In [ ]:
display(louvain_communities[6])
# Al Jazeera(lean left), African Diaspora news channel, France 24 English, Visual Politik EN

In [ ]:
df_tosave = nx.to_pandas_edgelist(sub_G,)
df_tosave.to_csv('data/louvain_filtered_graph.csv', sep=';', index=False)

In [ ]:
tool = range(243)
indices = list(louvain_partitions[2])
indices.sort()
indices = np.array(indices)
mapping = dict(zip(indices, tool))
mapping.get(i for i in indices)

In [ ]:
map_nodes = np.array(louvain_communities)
cols = ['red', 'yellow', 'orange', 'violet', 'blue', 'green', 'pink']
colors = np.array(243)
for i in range(len(louvain_communities)):
    for j in list(louvain_communities[i]):
        print(mapping.get(j))
        #print(np.array(louvain_communities[i]))
        colors[mapping.get(j)] = cols[i]
display(colors)

In [ ]:
nx.draw_networkx(sub_G, with_labels=False, node_size=10)

In [ ]:
forceatlas2 = ForceAtlas2()
posses = forceatlas2.forceatlas2_networkx_layout(sub_G)


In [ ]:
nx.draw_networkx(sub_G, with_labels=False,pos=posses, node_size=10, width=5e-2)